In [1]:
!pip install snowflake-snowpark-python[pandas]

     |████████████████████████████████| 1.2MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 54.0MB/s eta 0:00:01
     |████████████████████████████████| 870kB 55.4MB/s eta 0:00:01
     |████████████████████████████████| 747kB 47.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 14.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 54.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 11.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 58.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 60.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 11.7MB/s eta 0:00:01
     |████████████████████████████████| 143kB 52.9MB/s eta 0:00:01
     |████████████████████████████████| 450kB 56.1MB/s eta 0:00:01
     |████████████████████████████████| 112kB 55.4MB/s eta 0:00:01
 

In [1]:
from snowflake.snowpark import Session

In [6]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'].upper(), 
                         database=parser['Credentials']['database'].upper(),
                         schema=parser['Credentials']['schema'].upper(), 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [7]:
session = get_session()
# session.close()
session

connection_params =  {'user': 'PRAKHAR', 'password': 'Prak@apr24', 'account': 'ug94937.us-east4', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'TEST_DATABASE_PRAK', 'schema': 'TEST_SCHEMA_PRAK', 'role': 'PRAKHAR'}


OperationalError: 250001: Could not connect to Snowflake backend after 2 attempt(s).Aborting